In [51]:
import requests
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

from datetime import datetime
from dateutil import parser

import ast
import pandas as pd

In [40]:
link = 'https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?symbol=posglos&NrKadencji=8'
base_link = 'https://www.sejm.gov.pl/Sejm8.nsf/'

In [3]:
def gethtml(url):
    try:        
        with closing(requests.get(url, stream=False)) as resp:
            return resp
            if resp.status_code == 200 and resp.headers['content-type'] is not None:
                return resp             
            return None
    except Exception as e:
        print(e)
        #log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

In [32]:
def is_int(value):
  try:
    int(value)
    return True
  except:
    return False

In [4]:
result = gethtml(link)

In [42]:
bs = BeautifulSoup(result.text, 'lxml')
table = bs.find('table')
table_of_rows = []
for tr in table.findAll('tr'):
    values = [td.text for td in tr.findAll('td')]
    if(len(values) > 0):
        links = [lnk.get('href') for lnk in tr.findAll('a')]
        if(len(links) > 0):
            values.append(links[0])        
        #print(values)
        table_of_rows.append(values)

prev_voting_order = 0
for row in table_of_rows:
    if(is_int(row[0])):
        prev_voting_order=row[0]
    row[0]=prev_voting_order
    row[3] = base_link + '' + row[3]

In [46]:
df = pd.DataFrame(table_of_rows, columns = ['Nr_Pos', 'Data_Pos', 'Pos_Number', 'Link']) 

In [47]:
df.head()

,Nr_Pos,Data_Pos,Pos_Number,Link
0,86,11 września 2019 r.,88,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...
1,86,16 października 2019 r.,38,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...
2,85,31 lipca 2019 r.,24,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...
3,85,9 sierpnia 2019 r.,11,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...
4,85,30 sierpnia 2019 r.,38,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...


In [86]:
months = ['stycznia','lutego','marca','kwietnia','maja','czerwca','lipca','sierpnia', 'września', 'października','listopada', 'grudnia']

def date_conv(xdate):
    arr = xdate.replace('r.','').split()
    lstr = arr[0] + '-' + str(months.index(arr[1])+1)+'-'+arr[2]
    return datetime.strptime(lstr,'%d-%m-%Y')


df['Data_Pos_Parsed'] = df['Data_Pos'].map(date_conv)

In [85]:
df.sample(3)

,Nr_Pos,Data_Pos,Pos_Number,Link,Data_Pos_Parsed
70,53,6 grudnia 2017 r.,22,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2017-12-06
26,74,14 grudnia 2018 r.,66,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2018-12-14
22,76,16 stycznia 2019 r.,440,https://www.sejm.gov.pl/Sejm8.nsf/agent.xsp?sy...,2019-01-16


In [88]:
df.to_csv('sejm.csv', encoding='utf-8')